In [1]:
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize 
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [3]:
hero_df = pd.read_json('opendota_constants/heroes.json')
hero_df.rename(columns = {'id':'hero_id'}, inplace = True) 

In [4]:
hero_df.columns
hero_df = hero_df[['hero_id', 'localized_name']]

In [5]:
names = {}
with open('opendota_constants/heroes.json') as f:
    data = json.load(f)
    for h in data:
        names[h['id']] = h['localized_name']
print(names)

{1: 'Anti-Mage', 2: 'Axe', 3: 'Bane', 4: 'Bloodseeker', 5: 'Crystal Maiden', 6: 'Drow Ranger', 7: 'Earthshaker', 8: 'Juggernaut', 9: 'Mirana', 10: 'Morphling', 11: 'Shadow Fiend', 12: 'Phantom Lancer', 13: 'Puck', 14: 'Pudge', 15: 'Razor', 16: 'Sand King', 17: 'Storm Spirit', 18: 'Sven', 19: 'Tiny', 20: 'Vengeful Spirit', 21: 'Windranger', 22: 'Zeus', 23: 'Kunkka', 25: 'Lina', 26: 'Lion', 27: 'Shadow Shaman', 28: 'Slardar', 29: 'Tidehunter', 30: 'Witch Doctor', 31: 'Lich', 32: 'Riki', 33: 'Enigma', 34: 'Tinker', 35: 'Sniper', 36: 'Necrophos', 37: 'Warlock', 38: 'Beastmaster', 39: 'Queen of Pain', 40: 'Venomancer', 41: 'Faceless Void', 42: 'Wraith King', 43: 'Death Prophet', 44: 'Phantom Assassin', 45: 'Pugna', 46: 'Templar Assassin', 47: 'Viper', 48: 'Luna', 49: 'Dragon Knight', 50: 'Dazzle', 51: 'Clockwerk', 52: 'Leshrac', 53: "Nature's Prophet", 54: 'Lifestealer', 55: 'Dark Seer', 56: 'Clinkz', 57: 'Omniknight', 58: 'Enchantress', 59: 'Huskar', 60: 'Night Stalker', 61: 'Broodmother',

In [6]:
# another hero lookup method

#hero_lookup = dict(zip(hero_df['hero_id'], hero_df['localized_name']))
##hero_lookup[0] = 'Unknown'
#df_picks['hero_id'] = ['hero_id'].apply(lambda _id: hero_lookup[_id])

In [7]:
hero_lookup = dict(zip(hero_df['hero_id'], hero_df['localized_name']))

In [8]:
hero_name_df= hero_df[['hero_id','localized_name']]
hero_name_df.head(5)

,hero_id,localized_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [9]:
# possible way to flatten
#id_df = hero_df['id'].unstack()
#id_df.head()

## Cleaning:
- Filter relevant columns
- Flatten picks_bans to one row (index='match_id)
- Match hero_name to hero_id
- Rename picks/bans
- Join picks_bans with other matchdata columns

### 1. Cleaning Test: Single match

In [3]:
# match_id 4986461644 is game 4 ti9 grand finals
url = 'https://api.opendota.com/api/matches/4986461644'
r = requests.get(url)
data = r.json()

In [5]:
big_df = pd.json_normalize(data)
big_df.head()

,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,dire_score,dire_team_id,draft_timings,duration,engine,first_blood_time,game_mode,human_players,leagueid,lobby_type,match_seq_num,negative_votes,objectives,picks_bans,positive_votes,radiant_gold_adv,radiant_score,radiant_team_id,radiant_win,radiant_xp_adv,skill,start_time,teamfights,tower_status_dire,tower_status_radiant,version,replay_salt,series_id,series_type,players,patch,throw,loss,replay_url,cosmetics.647,cosmetics.4269,cosmetics.4538,cosmetics.4560,cosmetics.4580,cosmetics.5390,cosmetics.5406,cosmetics.5732,cosmetics.5967,cosmetics.6005,cosmetics.6432,cosmetics.6433,cosmetics.6434,cosmetics.6526,cosmetics.6687,cosmetics.6758,cosmetics.7580,cosmetics.7668,cosmetics.7935,cosmetics.7994,cosmetics.8203,cosmetics.8263,cosmetics.8964,cosmetics.9235,cosmetics.9526,cosmetics.9527,cosmetics.9528,cosmetics.9529,cosmetics.9530,cosmetics.9748,cosmetics.9762,cosmetics.11266,cosmetics.11321,cosmetics.12920,cosmetics.12947,cosmetics.12996,cosmetics.13235,cosmetics.13236,cosmetics.13262,cosmetics.13263,cosmetics.13264,cosmetics.13523,cosmetics.13541,cosmetics.13563,cosmetics.13564,cosmetics.13565,cosmetics.13566,cosmetics.16286,cosmetics.16674,cosmetics.17482,league.leagueid,league.ticket,league.banner,league.tier,league.name,radiant_team.team_id,radiant_team.name,radiant_team.tag,radiant_team.logo_url,dire_team.team_id,dire_team.name,dire_team.tag,dire_team.logo_url,all_word_counts.glhf,all_word_counts.gl,all_word_counts.hf,all_word_counts.gg,all_word_counts.ggg
0,4986461644,51,63,"[{'time': -100, 'type': 'chatwheel', 'key': '2...",308,11,2163,"[{'order': 1, 'pick': False, 'active_team': 2,...",1474,1,121,2,10,10749,1,4182489532,445,"[{'time': 121, 'type': 'CHAT_MESSAGE_FIRSTBLOO...","[{'is_pick': False, 'hero_id': 52, 'team': 0, ...",8963,"[0, 185, 193, 451, 40, 42, -130, -604, -1114, ...",20,2586976,True,"[0, -131, 5, -86, -27, 129, 963, 256, -290, -4...",None,1566733817,"[{'start': 1212, 'end': 1261, 'last_death': 12...",1926,1972,21,1194860475,360940,2,"[{'match_id': 4986461644, 'player_slot': 0, 'a...",41,1577,7103,http://replay308.valve.net/570/4986461644_1194...,132,129,129,129,129,132,128,130,4,131,131,131,131,131,132,132,129,132,131,128,4,132,132,1,3,3,3,3,3,129,129,129,3,2,0,3,0,0,0,0,0,3,2,2,2,2,2,2,132,2,10749,None,None,premium,The International 2019,2586976,OG,OG,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,2163,Team Liquid,Liquid,https://steamcdn-a.akamaihd.net/apps/dota2/ima...,2,2,3,6,1


## Normalize 'picks_bans'

Based on research, want to format 'picks_bans' object into single row such that each column we want turns into 22 columns. 

Reference: https://github.com/codexponent/dota2-draft-analysis/blob/master/README.md
Reference: https://medium.com/@shrinivasshetty21/deep-learning-with-keras-on-dota-2-statistics-9a937434f9e0

In [8]:
df_picks = json_normalize(big_df['picks_bans'][0])
#df_picks.drop(index=None, columns=['ord'], inplace=True)
df = df_picks[['match_id', 'order', 'is_pick', 'team', 'hero_id']]
df.dtypes

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


match_id    int64
order       int64
is_pick      bool
team        int64
hero_id     int64
dtype: object

Convert data frame to string, aggregate the desired columns into a single row s.t. all values in column expressed as a string 

In [1]:
df = df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)
df

NameError: name 'df' is not defined

In [14]:
#new = df['order'].str.split(' ', expand=True)
new1 = df['is_pick'].str.split(' ', expand=True)
new2 = df['team'].str.split(' ', expand=True)
new3 = df['hero_id'].str.split(' ', expand=True)
new1, new2, new3

(                0      1      2      3      4      5     6     7     8     9  \
 match_id                                                                       
 4986461644  False  False  False  False  False  False  True  True  True  True   
 
                10     11     12     13    14    15    16    17     18     19  \
 match_id                                                                       
 4986461644  False  False  False  False  True  True  True  True  False  False   
 
               20    21  
 match_id                
 4986461644  True  True  ,
             0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21
 match_id                                                                    
 4986461644  0  1  0  1  0  1  0  1  1  0  0  1  0  1  1  0  1  0  1  0  0  1,
              0   1   2   3   4    5   6   7   8   9  10  11   12  13  14   15  \
 match_id                                                                        
 4986461644  52  97  55  73  58 

In [15]:
print(list(new1))
print(list(new2))
print(list(new3))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


We've now 'flattened' each feature into a single row. Next step, setup and assign 22 column labels for each feature, and merge each of them. 

In [16]:
#new1.columns = col1
#new2.columns = col2
#new3.columns = col3

In [17]:
# Get column labels for the columns 
total_picks = 21
new1.columns = ['1is_pick', '2is_pick', '3is_pick', '4is_pick', '5is_pick', '6is_pick', '7is_pick','8is_pick','9is_pick','10is_pick','11is_pick','12is_pick','13is_pick','14is_pick','15is_pick','16is_pick','17is_pick','18is_pick','19is_pick','20is_pick','21is_pick', '22is_pick']
new2.columns = ['1team', '2team', '3team', '4team', '5team', '6team', '7team', '8team', '9team', '10team', '11team', '12team', '13team', '14team', '15team', '16team', '17team', '18team', '19team', '20team', '21team', '22team']
new3.columns = ['1hero_id', '2hero_id', '3hero_id', '4hero_id', '5hero_id', '6hero_id', '7hero_id', '8hero_id', '9hero_id', '10hero_id', '11hero_id', '12hero_id', '13hero_id', '14hero_id', '15hero_id', '16hero_id', '17hero_id', '18hero_id', '19hero_id', '20hero_id', '21hero_id',
               '22hero_id']

### PICKS_BANS FLATTENED

In [18]:
picks_df = new1.join(new2).join(new3)
picks_df.dtypes

1is_pick     object
2is_pick     object
3is_pick     object
4is_pick     object
5is_pick     object
6is_pick     object
7is_pick     object
8is_pick     object
9is_pick     object
10is_pick    object
11is_pick    object
12is_pick    object
13is_pick    object
14is_pick    object
15is_pick    object
16is_pick    object
17is_pick    object
18is_pick    object
19is_pick    object
20is_pick    object
21is_pick    object
22is_pick    object
1team        object
2team        object
3team        object
4team        object
5team        object
6team        object
7team        object
8team        object
9team        object
10team       object
11team       object
12team       object
13team       object
14team       object
15team       object
16team       object
17team       object
18team       object
19team       object
20team       object
21team       object
22team       object
1hero_id     object
2hero_id     object
3hero_id     object
4hero_id     object
5hero_id     object
6hero_id     object


## PUTTING IT ALL TOGETHER 

In [19]:
big_df = big_df[['match_id', 'dire_score', 'dire_team_id', 'duration', 'leagueid', 'match_seq_num', 'radiant_gold_adv', 'radiant_score', 'radiant_team_id', 'radiant_win', 'radiant_xp_adv', 'start_time', 'tower_status_dire', 'tower_status_radiant', 'version', 'series_id', 'series_type', 'patch']]
big_df

,match_id,dire_score,dire_team_id,duration,leagueid,match_seq_num,radiant_gold_adv,radiant_score,radiant_team_id,radiant_win,radiant_xp_adv,start_time,tower_status_dire,tower_status_radiant,version,series_id,series_type,patch
0,4986461644,11,2163,1474,10749,4182489532,"[0, 185, 193, 451, 40, 42, -130, -604, -1114, ...",20,2586976,True,"[0, -131, 5, -86, -27, 129, 963, 256, -290, -4...",1566733817,1926,1972,21,360940,2,41


In [20]:
match_df = big_df[['match_id','duration', 'leagueid','match_seq_num','series_id', 'series_type', 'patch', 'dire_team_id', 'radiant_team_id', 'radiant_win' ]]
match_df.dtypes

match_id           int64
duration           int64
leagueid           int64
match_seq_num      int64
series_id          int64
series_type        int64
patch              int64
dire_team_id       int64
radiant_team_id    int64
radiant_win         bool
dtype: object

In [21]:
match_df.join(picks_df, how='left')
#pd.concat([match_df, picks_df], sort=True)

#picks_df.join(match_df)

,match_id,duration,leagueid,match_seq_num,series_id,series_type,patch,dire_team_id,radiant_team_id,radiant_win,1is_pick,2is_pick,3is_pick,4is_pick,5is_pick,6is_pick,7is_pick,8is_pick,9is_pick,10is_pick,11is_pick,12is_pick,13is_pick,14is_pick,15is_pick,16is_pick,17is_pick,18is_pick,19is_pick,20is_pick,21is_pick,22is_pick,1team,2team,3team,4team,5team,6team,7team,8team,9team,10team,11team,12team,13team,14team,15team,16team,17team,18team,19team,20team,21team,22team,1hero_id,2hero_id,3hero_id,4hero_id,5hero_id,6hero_id,7hero_id,8hero_id,9hero_id,10hero_id,11hero_id,12hero_id,13hero_id,14hero_id,15hero_id,16hero_id,17hero_id,18hero_id,19hero_id,20hero_id,21hero_id,22hero_id
0,4986461644,1474,10749,4182489532,360940,2,41,2163,2586976,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Hmmmmm dunno if it's pandas running out of memory cache or something, but gettting NaN's for all of the RHS table joined

In [118]:
#new1.columns = col1
#new2.columns = col2
new3.columns = col3

FUCKED UP HERE cuts off 1 column

In [82]:
# trying to find faster way to rename columns 
for i in range(21):
    for col in new1.columns:
        new1.iloc[:,2:].add_prefix('{i+1}is_pick')
    list(new1)
new1

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
match_id,,,,,,,,,,,,,,,,,,,,,,
4986461644,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True


In [ ]:
heroes_list = []
for i, j in enumerate(heroes_dict):
    hero_name = heroes_dict[j]
    heroes_list.append(hero_name)

In [54]:
old = ["is_pick", "team", "hero_id"]
new = []
for i in range(1,22):
    for j in old_col:
        new_col = print(f"{i}{new_col}")
        new_col.append('i')
        print(new_col)

1None


AttributeError: 'NoneType' object has no attribute 'append'

In [146]:
#split_col = df[col].str.split(' ', expand=True)

NameError: name 'col' is not defined

In [143]:
df.iloc[:, lambda df: [1,2,3]].str.split(' ', expand=True)
#df2.iloc[:,-1:].str.split(' ', expand=True)

IndexError: positional indexers are out-of-bounds

In [138]:
df2[   '1is_pick', '1team_id', '1hero_id', 
       '2is_pick', '2team_id', '2hero_id', 
       '3is_pick', '3team_id', '3hero_id', 
       '4is_pick', '4team_id', '4hero_id', 
       '5is_pick', '5team_id', '5hero_id', 
       '6is_pick', '6team_id', '6hero_id', 
       '7is_pick', '7team_id', '7hero_id', 
       '8is_pick', '8team_id', '8hero_id', 
       '9is_pick', '9team_id', '9hero_id', 
       '10is_pick', '10team_id', '10hero_id', 
       '11is_pick', '11team_id', '11hero_id', 
       '12is_pick', '12team_id', '12hero_id',
       '13is_pick', '13team_id', '13hero_id', 
       '14is_pick', '14team_id', '14hero_id', 
       '15is_pick', '15team_id', '15hero_id', 
       '16is_pick', '16team_id', '16hero_id', 
       '17is_pick', '17team_id', '17hero_id',
       '18is_pick', '18team_id', '18hero_id', 
       '19is_pick', '19team_id', '19hero_id',
       '20is_pick', '20team_id', '20hero_id',
       '21is_pick', '21team_id', '21hero_id'] = df2.iloc[:,1:].str.split(' ', expand=True)

AttributeError: 'DataFrame' object has no attribute 'str'

In [ ]:
df['1is_pick',
    '2is_pick', 
    '3is_pick', 
    '4is_pick', 
    '5is_pick', 
    '6is_pick', 
    '7is_pick', 
    '8is_pick', 
    '9is_pick', 
    '10is_pick',
    '11is_pick',
    '12is_pick',
    '13is_pick',
    '14is_pick',
    '15is_pick',
    '16is_pick',
    '17is_pick',
    '18is_pick',
    '19is_pick',
    '20is_pick',
    '21is_pick'] = df['is_pick'].str.split(' ', expand=True)

In [126]:
is_pick = []

i=(1,22)
for i in is_pick:
    is_pick.append([i][is_pick])
    i += 1

print(is_pick)

[]


In [ ]:
df2 = df['1is_pick',
    '2is_pick', 
    '3is_pick', 
    '4is_pick', 
    '5is_pick', 
    '6is_pick', 
    '7is_pick', 
    '8is_pick', 
    '9is_pick', 
    '10is_pick',
    '11is_pick',
    '12is_pick',
    '13is_pick',
    '14is_pick',
    '15is_pick',
    '16is_pick',
    '17is_pick',
    '18is_pick',
    '19is_pick',
    '20is_pick',
    '21is_pick'] = df['is_pick'].str.split(' ', expand=True)

In [21]:
df2

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
match_id,,,,,,,,,,,,,,,,,,,,,,
4986461644,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True


In [67]:

is_picks_cols = list(map(lambda s: str(s)+'is_pick', str(s)+'team', str(s)+'hero_id'))
team_cols = list(map(lambda s: str(s)+'is_pick'
hero_id_cols = 

TypeError: <lambda>() takes 1 positional argument but 2 were given

In [78]:
range(21)

range(0, 21)

In [ ]:
for i, j, k in range(21): 
   picks_cols.append(list([i]+str(is_pick), [j]+str(team), [k]+str(hero_id)))

In [57]:
df2.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True


In [ ]:
df[['match_id', ]] 

= df['Vdf[['V','allele']] = df['V'].str.split('-',expand=True)'].str.split('-',expand=True)

In [ ]:
df = picks_df.astype(str)
df = df.groupby('match_id')['is_pick','team','hero_id'].agg(' '.join)

for m in df['match_id']:
    for col in df.columns:
        
    

for col in df.columns:
    

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True


In [107]:
flat_df.dropna(inplace=True)
new = flat_df[['is_pick','team','hero_id'].str.split(' ', expand=True)

SyntaxError: unexpected EOF while parsing (<ipython-input-107-8563e9b36163>, line 2)

In [ ]:
# NEW DATA FRAME
df_picks = pd.DataFrame['match_id' '1is_pick', '1team', '1hero_id', 
       '2is_pick', '2team', '2hero_id', 
       '3is_pick', '3team', '3hero_id', 
       '4is_pick', '4team', '4hero_id', 
       '5is_pick', '5team', '5hero_id', 
       '6is_pick', '6team', '6hero_id', 
       '7is_pick', '7team', '7hero_id', 
       '8is_pick', '8team', '8hero_id', 
       '9is_pick', '9team', '9hero_id', 
       '10is_pick', '10team', '10hero_id', 
       '11is_pick', '11team', '11hero_id', 
       '12is_pick', '12team', '12hero_id',
       '13is_pick', '13team', '13hero_id', 
       '14is_pick', '14team', '14hero_id', 
       '15is_pick', '15team', '15hero_id', 
       '16is_pick', '16team', '16hero_id', 
       '17is_pick', '17team', '17hero_id',
       '18is_pick', '18team', '18hero_id', 
       '19is_pick', '19team', '19hero_id',
       '20is_pick', '20team', '20hero_id',
       '21is_pick', '21team', '21hero_id']= flat_df['match_id']['is_pick']['team']['hero_id'].str.split(' ', expand=True)

In [ ]:
df[['1pick', '2pick', '3pick', '4pick', '5pick']] = df['Picks'].str.split(' ', expand=True)

In [ ]:
'1is_pick', '1team', '1hero_id', 
       '2is_pick', '2team', '2hero_id', 
       '3is_pick', '3team', '3hero_id', 
       '4is_pick', '4team', '4hero_id', 
       '5is_pick', '5team', '5hero_id', 
       '6is_pick', '6team', '6hero_id', 
       '7is_pick', '7team', '7hero_id', 
       '8is_pick', '8team', '8hero_id', 
       '9is_pick', '9team', '9hero_id', 
       '10is_pick', '10team', '10hero_id', 
       '11is_pick', '11team', '11hero_id', 
       '12is_pick', '12team', '12hero_id',
       '13is_pick', '13team', '13hero_id', 
       '14is_pick', '14team', '14hero_id', 
       '15is_pick', '15team', '15hero_id', 
       '16is_pick', '16team', '16hero_id', 
       '17is_pick', '17team', '17hero_id',
       '18is_pick', '18team', '18hero_id', 
       '19is_pick', '19team', '19hero_id',
       '20is_pick', '20team', '20hero_id',
       '21is_pick', '21team', '21hero_id'],

In [ ]:
df_picks.groupby('match_id')['team','is_pick','hero_id'].agg(' '.join)

In [101]:
# def unstack_simplify(df_picks):
#    return df_picks.unstack().iloc[10:].reset_index(drop=True)

In [27]:
x = len(df_picks['order'])
print(x)

22


is_pick = 0 if radiant, 1 if dire 

In [45]:
df_picks[['match_id', 'is_pick', 'order']].pivot_table(columns=['match_id','order']).shape

(22,)

In [172]:
merge_df = pd.merge(df_picks, hero_df, on = 'hero_id', how = 'left')
merge_df.set_index(['match_id'],inplace = True)
merge_df.T
#merge_df.pivot_table(columns=['match_id'])

match_id,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644,4986461644
is_pick,False,False,False,False,False,False,True,True,True,True,False,False,False,False,True,True,True,True,False,False,True,True
hero_id,52,97,55,73,58,103,91,66,57,19,82,95,106,74,99,102,107,98,53,11,72,21
team,0,1,0,1,0,1,0,1,1,0,0,1,0,1,1,0,1,0,1,0,0,1
order,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
localized_name,Leshrac,Magnus,Dark Seer,Alchemist,Enchantress,Elder Titan,Io,Chen,Omniknight,Tiny,Meepo,Troll Warlord,Ember Spirit,Invoker,Bristleback,Abaddon,Earth Spirit,Timbersaw,Nature's Prophet,Shadow Fiend,Gyrocopter,Windranger


In [108]:
def unstack_simplify(merge_df):
    return merge_df.unstack(level=0) #.iloc[10:].reset_index(drop=True)

In [176]:
merge_df

,is_pick,hero_id,team,order,localized_name
match_id,,,,,
4986461644,False,52,0,0,Leshrac
4986461644,False,97,1,1,Magnus
4986461644,False,55,0,2,Dark Seer
4986461644,False,73,1,3,Alchemist
4986461644,False,58,0,4,Enchantress
4986461644,False,103,1,5,Elder Titan
4986461644,True,91,0,6,Io
4986461644,True,66,1,7,Chen
4986461644,True,57,1,8,Omniknight


In [178]:
id_df = merge_df.unstack()
id_df.head()

         match_id  
is_pick  4986461644    False
         4986461644    False
         4986461644    False
         4986461644    False
         4986461644    False
dtype: object

In [ ]:
x=len(order)

In [166]:
df2 = df_picks.T

In [30]:
cols = list(df_picks_bans.columns.values)
cols

['is_pick', 'hero_id', 'team', 'order', 'match_id']

### Flattening picks data 

Approuch:
- write a function that for each match_id, takes the 'is_pick' and 'team' columns from 'picks_bans'  and:
    - for that match and puts them in a new dataframe along row axis with column names 1is_pick, etc... 

In [12]:
# IDEAL DATAFRAME OUTPUT

Ideal_df.columns(['match_id', 'radiant_win', 'duration', 
       '1is_pick', '1team_id', '1hero_id', 
       '2is_pick', '2team_id', '2hero_id', 
       '3is_pick', '3team_id', '3hero_id', 
       '4is_pick', '4team_id', '4hero_id', 
       '5is_pick', '5team_id', '5hero_id', 
       '6is_pick', '6team_id', '6hero_id', 
       '7is_pick', '7team_id', '7hero_id', 
       '8is_pick', '8team_id', '8hero_id', 
       '9is_pick', '9team_id', '9hero_id', 
       '10is_pick', '10team_id', '10hero_id', 
       '11is_pick', '11team_id', '11hero_id', 
       '12is_pick', '12team_id', '12hero_id',
       '13is_pick', '13team_id', '13hero_id', 
       '14is_pick', '14team_id', '14hero_id', 
       '15is_pick', '15team_id', '15hero_id', 
       '16is_pick', '16team_id', '16hero_id', 
       '17is_pick', '17team_id', '17hero_id',
       '18is_pick', '18team_id', '18hero_id', 
       '19is_pick', '19team_id', '19hero_id',
       '20is_pick', '20team_id', '20hero_id',
       '21is_pick', '21team_id', '21hero_id'],
      dtype='object')

NameError: name 'Index' is not defined

In [ ]:
# 
big_df = json_normalize(data)
picks_df = json_normalize(big_df['picks_bans'][0])

picks_cols = list(map(lambda s: [i] + 'is_pick', df_picks ))

X_pick = np.zeros((data.shape[0], N))
# for p in xrange(5):

Def picks()
for i in picks_df[match_id]:
    for p in len(picks_df['order']):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
    

In [ ]:
# Prepare column names for the clean picks dataframe
# 1is_pick	1team_id	1hero_id
df_picks_bans.columns = df_picks.columns[-1:] + df_team.columns[] + df_hero_id.columns[]
df_picks_cols = list(map(lambda s: [i] + 'is_pick', df_picks ))

In [ ]:
df_pick_bans.columns = df_picks_bans.columns[-1:] + df_picks_bans.columns[:-1]
df_picks_bans = df_picks_bans[columns]

In [ ]:
pd.json_normalize(data, 'picks_bans')

In [ ]:
df = data['id']['localized_name']

In [ ]:
# Attempt 2: 
# original code
# source :https://github.com/dotascience/kaggle-dota2-win-prediction/blob/master/ContestDescription.ipynb
X_pick = np.zeros((data.shape[0], N))
# for p in xrange(5):

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
# adapting
X_pick = np.zeros((data.shape[0], N))

for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
for i, match_id in enumerate(data.index):
    for p in xrange(5):
        X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

In [ ]:
# picks_ban parse of matchdata as dataframe


# ref: https://kite.com/python/answers/how-to-append-rows-to-a-pandas-dataframe-using-a-for-loop-in-python
#pandas.Dataframe.append(data, ignore_index=None)

# list of match_id's for the event

input_file = open('opendota_matches/match_list/league_id_10979_starladder_s2.json') 
matches = json.load(input_file) # !! list of match_id's for given event
input_file.close()

# JSON file with match data from get request
with open('opendota_matches/matches/league_id_10979_starladder_s2_matchdata.json', 'r') as matchdata:
    JSONcontent = json.load(matchdata) # matchdata now deserialized json object 
#matchdata.close()

data = json_normalize(JSONcontent) #json object --> table 


# GO THROUGH MATCH DATA AND EXTRACT PICKS_BANS FEATURE AND WRITE PICKS_BANS DATA INTO 1 LINE PER MATCH
# Nested loop at end should give me 'picks_bans' data in a single line formatted as:
# "Match detail columns" + 1is_pick, 1team, 1hero_id, 2is_pick,...
results=[]
with open('picks_bans/leagueid_10979_starladder_picks_bans.csv', 'a') as picks:
    for m in matches:
        #curmatch = data["match_id"][0]["picks_bans"].index(m) # !! Stuck @ this step 
        curmatch = data[['match_id','leagueid','series_id','series_type','duration', 'dire_team_id','radiant_team_id','radiant_win','picks_bans']]
        print(f" processsing match: {m['match_id']}")
        #if 'picks_ban' in curmatch: 
        for i in range(picks_df[order]) # 'order' = order of picks from 0-21 
            print( 'm' + str(match_id))
            picks.write(str(match_id) + ",") # match_id
            picks.write(str(dire_team_id) + ",") # team playing on dire side 
            picks.write(str(radiant_team_id) + ",") # team playin on radiant side
            picks.write(str(curmatch['radiant_win']) + ",") # match result 
            picks.write(str(curmatch['duration'])) # length of match 
            picklist = curmatch['picks_bans'] # define picks_bans feature as 'picklist'
            for i in range(len(picklist)):
                picks.write("," + str(picklist[i]['is_pick']) + "," + str(picklist[i]['team']) + "," + str(picklist[i]['hero_id'])) 
            picks.write("\n")

In [ ]:
# semi-modified version of picks_ban parse in json 

# list of match_id's for the event

input_file = open('opendota_matches/match_list/league_id_10979_starladder_s2.json') 
matches = json.load(input_file) # !! list of match_id's for given event
input_file.close()

# JSON file with match data from get request
with open('opendota_matches/matches/league_id_10979_starladder_s2_matchdata.json', 'r') as matchdata:
    JSONcontent = json.load(matchdata) # matchdata now deserialized json object 
#matchdata.close()

data = json_normalize(JSONcontent) #json object --> table 


# GO THROUGH MATCH DATA AND EXTRACT PICKS_BANS FEATURE AND WRITE PICKS_BANS DATA INTO 1 LINE PER MATCH
# Nested loop at end should give me 'picks_bans' data in a single line formatted as:
# "Match details", + 1is_pick, 1team, 1hero_id, 2is_pick,...
results=[]
with open('picks_bans/leagueid_10979_starladder_picks_bans.csv', 'a') as picks:
    for m in matches:
        #curmatch = data["match_id"][0]["picks_bans"].index(m) # !! Stuck @ this step 
        curmatch = data[['match_id','leagueid','series_id','series_type','duration', 'dire_team_id','radiant_team_id','radiant_win','picks_bans']]
        print(f" processsing match: {m['match_id']}")
        if 'picks_ban' in curmatch: 
            print( 'm' + str(match_id))
            picks.write(str(match_id) + ",") # match_id
            picks.write(str(dire_team_id) + ",") # team playing on dire side 
            picks.write(str(radiant_team_id) + ",") # team playin on radiant side
            picks.write(str(curmatch['radiant_win']) + ",") # match result 
            picks.write(str(curmatch['duration'])) # length of match 
            picklist = curmatch['picks_bans'] # define picks_bans feature as 'picklist'
            for i in range(len(picklist)):
                picks.write("," + str(picklist[i]['is_pick']) + "," + str(picklist[i]['team']) + "," + str(picklist[i]['hero_id'])) 
            picks.write("\n")